In [1]:
import numpy as np
import tensorflow as tf
import cv2
from tqdm import tqdm
import processing
import argparse
from model import build_model
import os
import pandas as pd
import pickle
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import keras.backend.tensorflow_backend as KTF

Using TensorFlow backend.


In [3]:
if '__file__' not in globals():
    CURRENT_DIR = os.getcwd()
else:
    CURRENT_DIR = os.path.dirname(os.path.realpath(__file__))

In [4]:
print(CURRENT_DIR)

/home/ubuntu/repositories/speed-challenge


In [17]:
MINI_BATCH_SIZE = 16
EPOCHS = 50
VERSION = 2

SIZE = (100, 100)
CHANNEL = 2
WIDTH = SIZE[0]
HEIGHT = SIZE[1]

DATA_PATH =  os.path.join(CURRENT_DIR, 'data')
TRAIN_VIDEO = os.path.join(DATA_PATH, 'train.mp4')
TEST_VIDEO = os.path.join(DATA_PATH, 'test.mp4')
PREPARED_DATA_PATH = os.path.join(DATA_PATH, 'prepared-data')
PREPARED_IMGS_TRAIN = os.path.join(PREPARED_DATA_PATH, 'train_imgs')
FLOW_IMGS_TRAIN = os.path.join(PREPARED_DATA_PATH, 'flow_train_imgs')
PREPARED_IMGS_TEST = os.path.join(PREPARED_DATA_PATH, 'test_imgs')
FLOW_IMGS_TEST = os.path.join(PREPARED_DATA_PATH, 'flow_test_imgs')
SAVES = os.path.join(CURRENT_DIR, 'saves')
MODEL_SAVE_PATH = os.path.join(CURRENT_DIR, 'saves', str(VERSION), str(EPOCHS))
WEIGHTS_PATH = os.path.join(MODEL_SAVE_PATH, 'weights.{epoch:02d}-{val_loss:.2f}.hdf5')
HISTORY_PATH = os.path.join(MODEL_SAVE_PATH, 'history.p')
TENSORBOARD = os.path.join(MODEL_SAVE_PATH, 'tensorboard')

In [18]:
# Create the save folder
if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH)

train_meta = pd.read_csv(os.path.join(PREPARED_DATA_PATH, "train_flow_meta.csv"))
print('shape: ', train_meta.shape)    

shape:  (20399, 2)


In [19]:
train_meta.head()

,flow_path,speed
0,/home/ubuntu/repositories/speed-challenge/data...,28.105569
1,/home/ubuntu/repositories/speed-challenge/data...,28.106048
2,/home/ubuntu/repositories/speed-challenge/data...,28.118466
3,/home/ubuntu/repositories/speed-challenge/data...,28.119823
4,/home/ubuntu/repositories/speed-challenge/data...,28.098907


In [21]:
#Load the data
X = np.empty((len(train_meta), HEIGHT, WIDTH, CHANNEL))
Y = np.empty((len(train_meta), 1))
for index, row in tqdm(train_meta.iterrows()):
    frame = cv2.imread(row["flow_path"])
    frame = cv2.resize(frame, SIZE, interpolation=cv2.INTER_AREA)
    #Normalize
    frame = frame / 40
    #Drop the useless channel
    frame = frame[:,:,[1,2]]
    X[index,:,:,:] = frame
    Y[index] = row["speed"]
#Shuffle the data
randomize = np.arange(len(train_meta))
np.random.shuffle(randomize)
X = X[randomize]
Y = Y[randomize]

20399it [00:17, 1184.07it/s]


In [22]:
print(X.shape)
print(Y.shape)

(20399, 100, 100, 2)
(20399, 1)


In [23]:
modelCheckpoint = ModelCheckpoint(WEIGHTS_PATH,
                                      monitor='val_loss',
                                      save_best_only=False,
                                      mode='min',
                                      verbose=1,
                                      save_weights_only=True)

tensorboard = TensorBoard(log_dir=TENSORBOARD, histogram_freq=0,
                              write_graph=True, write_images=True)

callbacks_list = [modelCheckpoint, tensorboard]


model = build_model(HEIGHT, WIDTH, CHANNEL)


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 48, 48, 24)        1224      
_________________________________________________________________
elu_1 (ELU)                  (None, 48, 48, 24)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 22, 22, 36)        21636     
_________________________________________________________________
elu_2 (ELU)                  (None, 22, 22, 36)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 9, 9, 48)          43248     
_________________________________________________________________
elu_3 (ELU)                  (None, 9, 9, 48)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 9, 48)          0         
__________

In [25]:
history = model.fit(
        X,
        Y,
        batch_size=MINI_BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=callbacks_list,
        verbose=1,
        validation_split=0.3)

pickle.dump(history.history, open(HISTORY_PATH, "wb"))

Train on 14279 samples, validate on 6120 samples
Epoch 1/40
14279/14279 [==============================] - 12s 822us/step - loss: 49.7818 - val_loss: 14.8037

Epoch 00001: saving model to /home/ubuntu/repositories/speed-challenge/saves/2/40/weights.01-14.80.hdf5
Epoch 2/40
14279/14279 [==============================] - 10s 729us/step - loss: 27.1256 - val_loss: 10.3375

Epoch 00002: saving model to /home/ubuntu/repositories/speed-challenge/saves/2/40/weights.02-10.34.hdf5
Epoch 3/40
14279/14279 [==============================] - 10s 728us/step - loss: 21.2270 - val_loss: 8.9068

Epoch 00003: saving model to /home/ubuntu/repositories/speed-challenge/saves/2/40/weights.03-8.91.hdf5
Epoch 4/40
14279/14279 [==============================] - 10s 730us/step - loss: 18.8293 - val_loss: 12.9062

Epoch 00004: saving model to /home/ubuntu/repositories/speed-challenge/saves/2/40/weights.04-12.91.hdf5
Epoch 5/40
14279/14279 [==============================] - 10s 730us/step - loss: 16.9139 - val_lo

KeyboardInterrupt: 